<a href="https://colab.research.google.com/github/BaghasWahyu/UAS-Machine-Learning/blob/main/Torrent_To_Google_Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader 

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

In [8]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent

     |████████████████████████████████| 1.6MB 36.0MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (57.0.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 2.4 MB 34.1 MB/s 


### Install libtorrent and Initialize Session

In [9]:
!apt install python-libtorrent
 
import libtorrent as lt
 
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-libtorrent is already the newest version (1.1.5-1build1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


### Mount Google Drive
To stream files we need to mount Google Drive.

In [10]:
from google.colab import drive
 
drive.mount("/content/drive")

Mounted at /content/drive


### Add From Torrent File
You can run this cell to add more files as many times as you want

In [11]:
from google.colab import files
 
source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

Saving [Izayoi Seishin, Yamasaki Masato] Boshi Double Soukan [English] [Tadanohito + Sei-Jin] [decensored].torrent to [Izayoi Seishin, Yamasaki Masato] Boshi Double Soukan [English] [Tadanohito + Sei-Jin] [decensored].torrent


### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [12]:
params = {"save_path": "/content/drive/My Drive/Torrent"}
 
while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

Enter Magnet Link Or Type Exit: Exit


### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [13]:
import time
from IPython.display import display
import ipywidgets as widgets
 
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]
 
layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)
 
while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
 
            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

[Izayoi Seishin, Yamasaki Masato] Boshi Double Soukan [English] [Tadanohito Sei-Jin] [decensored] complete
